In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
%matplotlib inline

In [ ]:
dict_path = '../input/'
train_df = pd.read_csv(dict_path + "train.csv")
test_df = pd.read_csv(dict_path + "sample_submission.csv")

data_folder = Path(dict_path)
train_img = ImageList.from_df(train_df, path=data_folder, folder='train/train')
test_img = ImageList.from_df(test_df, path=data_folder, folder='test/test')

In [ ]:
type(train_img)

In [ ]:
transformations = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, 
                                 max_zoom=1.1, max_lighting=0.2, max_warp=0.2, 
                                 p_affine=0.75, p_lighting=0.75)

In [ ]:
train_img = train_img.split_by_rand_pct(0.01)
train_img = train_img.label_from_df()
train_img = train_img.add_test(test_img)
train_img = train_img.transform(transformations, size=128)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
train_img = train_img.databunch(path='.', bs=64, device=torch.device(device))
train_img = train_img.normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(train_img, models.densenet161, metrics=[error_rate, accuracy])

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 3e-02
learn.fit_one_cycle(5, slice(lr))

In [ ]:
preds,_ = learn.get_preds(ds_type=DatasetType.Test) 
test_df.has_cactus = preds.numpy()[:, 0]

In [ ]:
test_df.to_csv('submission.csv', index=False)